In [1]:
import numpy as np
import pandas as pd
from astropy import constants

In [2]:
# Load in John Southworth's well-studied TEPCAT as csv file
data_tepcat = pd.read_csv('http://www.astro.keele.ac.uk/jkt/tepcat/allplanets-csv.csv', sep = ',', index_col = [0])

# Strip index (system names) of white space
data_tepcat.index = data_tepcat.index.str.strip()

# Replace underscores in names with spaces to match NASA Exoplanet Archive naming convention (if wanting to combine)
data_tepcat.index = data_tepcat.index.str.replace("_"," ")

# Strip column names of white space
data_tepcat.columns = data_tepcat.columns.str.strip()

In [3]:
# Sort table by density as an example
# data_tepcat.sort_values(by=['rho_b'],ascending=True)[(data_tepcat.rho_b >= 0)]

In [4]:
# What columns do we have?
data_tepcat.columns

Index(['Teff', 'err', 'err .1', '[Fe/H]', 'erru', 'errd', 'M_A', 'errup',
       'errdn', 'R_A', 'errup .1', 'errdn .1', 'loggA', 'errup .2', 'errdn .2',
       'rho_A', 'errup', 'errdn', 'Period', 'e', 'errup', 'errdown', 'a(AU)',
       'errup', 'errdown', 'M_b', 'errup', 'errdn', 'R_b', 'errup .1',
       'errdn .1', 'g_b', 'errup .2', 'errdn .2', 'rho_b', 'errup .3',
       'errdn .3', 'Teq', 'err .2', 'err .3', 'Discovery_reference',
       'Recent_reference'],
      dtype='object')

Replace semimajor column header

In [5]:
data_tepcat.rename(columns={'a(AU)':'semimajor'},inplace=True)

Remove planets which have no measured mass or radius

In [6]:
# Drop without measured mass
data_tepcat.drop(data_tepcat[data_tepcat.M_b == 0].index, inplace=True)

# Drop without measured radius
data_tepcat.drop(data_tepcat[data_tepcat.R_b == 0].index, inplace=True)

In [7]:
# Load in TEPCAT observables
data_obs = pd.read_csv('http://www.astro.keele.ac.uk/jkt/tepcat/observables.csv', sep = ',', index_col = [0])

# Strip index (system names) of white space
data_obs.index = data_obs.index.str.strip()

# Replace underscores in names with spaces to match NEA naming convention
data_obs.index = data_obs.index.str.replace("_"," ")

# Strip column names of white space
data_obs.columns = data_obs.columns.str.strip()

Merge the 2 TEPCAT tables so we have planet parameters and observability parameters

In [8]:
data = pd.merge(data_tepcat,data_obs,left_index=True,right_index=True)

Calcuate surface gravity afresh for those planets with massing values.

In [9]:
surface_gravity = (constants.G.value*data.M_b*constants.M_jup.value)/(data.R_b*constants.R_jup.value)**2
data = data.assign(g_b_JK=pd.Series(surface_gravity).values)

Calculate scale height using literature values

In [10]:
H = (constants.k_B.value*data.Teq)/(2.3*constants.m_p.value*data.g_b*1e3) 
data = data.assign(H=pd.Series(H).values)

Calculate scale height using my gravity and Teq values

In [11]:
# assume albedo of 0.3
equilibrium_temperature = (1 - 0.3)**(1/4)*data.Teff*np.sqrt((data.R_A*constants.R_sun.value)/(data.semimajor*constants.au.value))/np.sqrt(2)

data = data.assign(Teq_JK=pd.Series(equilibrium_temperature).values)

/Users/james/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  


In [12]:
# Using my own calc for surface gravity
H_JK = (constants.k_B.value*data.Teq_JK)/(2.3*constants.m_p.value*data.g_b_JK*1e3) 
data = data.assign(H_JK=pd.Series(H_JK).values)

Transmission signal per scale height in ppm

In [13]:
trans_signal = (2*data.H*1e3*data.R_b*constants.R_jup.value)/((data.R_A*constants.R_sun.value)**2)*1e6

data = data.assign(trans_signal=pd.Series(trans_signal).values)


# Using my own calc for surface gravity
trans_signal_JK = (2*data.H_JK*1e3*data.R_b*constants.R_jup.value)/((data.R_A*constants.R_sun.value)**2)*1e6

data = data.assign(trans_signal_JK=pd.Series(trans_signal_JK).values)

In [14]:

def save_xls(list_dfs,snames,xls_path):
    writer = pd.ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer,sheet_name=snames[n])
    writer.save()
    

save_xls([data],['all'],'all_transiting_planets.xls')

In [127]:
# Remove fainter than V=13
data.drop(data[data.Vmag > 13.5].index, inplace=True)

# Remove southern targets
data.drop(data[data.Decd <= -15].index, inplace=True)

# Remove small signals
# data.drop(data[data.trans_signal_JK <= 100].index, inplace=True)

kept_planets = data[data.trans_signal_JK >= 200]

In [128]:
kept_planets.sort_values(by=['trans_signal_JK'],ascending=False,inplace=True)#[(data.trans_signal >= 0)]

/Users/james/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [129]:
kept_planets

,Teff,err,err .1,[Fe/H],erru,errd,M_A,errup,errdn,R_A,...,T0err,Period(day),Perioderr,Ephemeris_reference,g_b_JK,H,Teq_JK,H_JK,trans_signal,trans_signal_JK
WASP-177,5017,70,70,0.250,0.040,0.040,0.876,0.038,0.038,0.885,...,0.000280,3.071722,1.000000e-06,arXiv:1903.06622,5.043884,872.018630,1046.503100,744.617006,519.684477,443.758752
WASP-107,4430,120,120,0.020,0.100,0.100,0.691,0.050,0.050,0.657,...,0.000011,5.721492,4.600000e-07,2017MNRAS.469.1622M,3.454765,854.824286,673.490778,699.633513,540.581924,442.440905
WASP-127,5750,100,100,-0.180,0.060,0.060,1.080,0.030,0.030,1.390,...,0.000068,4.178070,5.700000e-07,arXiv:1805.11744,2.377097,2347.857745,1311.237560,1979.666254,491.819996,414.692692
WASP-039,5485,50,50,0.010,0.090,0.090,0.913,0.046,0.046,0.939,...,0.000140,4.055280,7.000000e-07,2016ApJ...827...19F,4.257756,982.305258,1066.924296,899.311908,420.948789,385.383521
WASP-052,5000,100,100,0.030,0.120,0.120,0.804,0.050,0.050,0.786,...,0.000160,1.749781,5.200000e-07,2017MNRAS.465..843M,6.851776,688.957923,1202.653456,629.933763,412.802305,377.436851
WASP-183,5313,72,72,-0.310,0.040,0.040,0.784,0.038,0.038,0.871,...,0.002400,4.111777,5.100000e-06,arXiv:1903.06622,5.758171,766.775509,1016.184172,633.352294,438.927401,362.551585
HAT-P-32,6269,64,64,-0.040,0.080,0.080,1.182,0.049,0.049,1.225,...,0.000096,2.150008,1.000000e-07,arXiv:1712.00415,6.072804,1077.258625,1648.126314,973.999607,383.221080,346.487995
HAT-P-12,4665,45,45,-0.200,0.090,0.090,0.691,0.023,0.023,0.679,...,0.000036,3.213058,1.300000e-07,arXiv:1810.02172,5.899030,581.896298,873.567630,531.463531,342.661248,312.962907
HAT-P-48,5946,50,50,0.020,0.080,0.080,1.099,0.041,0.041,1.223,...,0.000290,4.408650,8.000000e-06,arXiv:1606.04556,3.255364,1507.546207,1244.646512,1372.157376,336.762526,306.518754
HAT-P-51,5449,50,50,0.270,0.080,0.080,0.976,0.028,0.028,1.041,...,0.000400,4.218028,5.900000e-06,2015AJ....150..168H,4.581176,934.046064,1089.152857,853.236401,329.237122,300.752937


Save to excel spreadsheet

In [130]:

    
out_file = 'tepcat_planets.xls'

sheet_names = ['all']

df_list = [kept_planets]

save_xls(df_list,sheet_names,out_file)

Save names to file

In [131]:
kept_planets.index

Index(['WASP-177', 'WASP-107', 'WASP-127', 'WASP-039', 'WASP-052', 'WASP-183',
       'HAT-P-32', 'HAT-P-12', 'HAT-P-48', 'HAT-P-51', 'HAT-P-47', 'HAT-P-19',
       'HAT-P-65', 'HAT-P-18', 'WASP-181', 'TrES-4', 'HAT-P-44', 'WASP-069',
       'Qatar-10', 'WASP-021', 'WASP-076', 'WASP-153'],
      dtype='object')

In [132]:
names = open('planet_names.txt','w')

for i in kept_planets.index:
    names.write("%s \n"%i)
    
names.close()